# Introduction

Make sure you have 'LM Studio' running locally. 'LM Studio' loaded models listen on:
`http://localhost:1234/v1/chat/completions`

# Install required Python dependencies

In [1]:
# LiteLLM (https://github.com/BerriAI/litellm)

# Uncomment the below line to install litellm
# !pip install litellm

# Code samples

## A simple agent who lists files in a directory

In [2]:
#
# Imports ...
#
import os
import json
import sys
from litellm import completion
from typing import List, Dict

#
# LM Studio URL 
#
os.environ['LM_STUDIO_API_BASE'] = "http://localhost:1234/v1"
os.environ['LM_STUDIO_API_KEY'] = "42" # Not really used. Set it to a non empty value

In [3]:
#
# Helper functions
#

def extract_markdown_block(response: str, block_type: str = "json") -> str:
    """Extract code block from response"""

    if not '```' in response:
        return response

    code_block = response.split('```')[1].strip()

    if code_block.startswith(block_type):
        code_block = code_block[len(block_type):].strip()

    return code_block

def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get a response."""
    response = completion(
        model="lm_studio/lmstudio",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content.strip()

def parse_action(response: str) -> Dict:
    """Parse the LLM response into a structured action dictionary."""
    try:
        response = extract_markdown_block(response, "action")
        response_json = json.loads(response)
        if "tool_name" in response_json and "args" in response_json:
            return response_json
        else:
            return {"tool_name": "error", "args": {"message": "You must respond with a JSON tool invocation."}}
    except json.JSONDecodeError:
        return {"tool_name": "error", "args": {"message": "Invalid JSON response. You must respond with a JSON tool invocation."}}

In [4]:
#
# Define the tools/actions available to the agent
#

def list_files(directory_name: str) -> List[str]:
    """List files in the current directory."""
    # TODO: Add check if directory does not exixts
    return os.listdir(os.path.expanduser(directory_name))

def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"

In [5]:
# These agent rules are specified in the system message and play
# a critical role in ensuring the agent interacts predictably
# and within its defined boundaries.

#
# Define system instructions (Agent Rules)
#
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

Available tools:

```json
{
    "list_files": {
        "description": "Lists all files in the specified directory.",
        "parameters": {
            "directory_name": {
                "type": "string",
                "description": "The name of the drectory to list files."
            }
        }
    },
    "read_file": {
        "description": "Reads the content of a file.",
        "parameters": {
            "file_name": {
                "type": "string",
                "description": "The name of the file to read."
            }
        }
    },
    "terminate": {
        "description": "Ends the agent loop and provides a summary of the task.",
        "parameters": {
            "message": {
                "type": "string",
                "description": "Summary message to return to the user."
            }
        }
    }
}
```

If a user asks about files, documents, or content, first list the files before reading them.

When you are done, terminate the conversation by using the "terminate" tool and I will provide the results to the user.

Important!!! Every response MUST have an action.
You must ALWAYS respond in this format:

<Stop and think step by step. Parameters map to args. Insert a rich description of your step by step thoughts here.>

```action
{
    "tool_name": "insert tool_name",
    "args": {...fill in any required arguments here...}
}
```"""
}]

In [6]:
# Initialize agent parameters
iterations = 0
max_iterations = 10

user_task = input("What would you like me to do? ")

memory = [{"role": "user", "content": user_task}]

##################
# The Agent Loop #
##################
while iterations < max_iterations:
    # 1. Construct prompt: Combine agent rules with memory
    prompt = agent_rules + memory

    # 2. Generate response from LLM
    print("Agent thinking...")
    response = generate_response(prompt)
    print(f"Agent response: {response}")

    # 3. Parse response to determine action
    action = parse_action(response)
    result = "Action executed"

    if action["tool_name"] == "list_files":
        result = {"result": list_files(action["args"]["directory_name"])}
    elif action["tool_name"] == "read_file":
        result = {"result": read_file(action["args"]["file_name"])}
    elif action["tool_name"] == "error":
        result = {"error": action["args"]["message"]}
    elif action["tool_name"] == "terminate":
        print(action["args"]["message"])
        break
    else:
        result = {"error": "Unknown action: " + action["tool_name"]}

    print(f"Action result: {result}")

    # 5. Update memory with response and results
    memory.extend([
        {"role": "assistant", "content": response},
        {"role": "user", "content": json.dumps(result)}
    ])

    # 6. Check termination condition
    if action["tool_name"] == "terminate":
        break

    iterations += 1

What would you like me to do?  list files in ~/Downloads


Agent thinking...
Agent response: Okay, I need to list the files in the Downloads directory. This will help me understand what files are available and potentially answer the user's questions about them.

```action
{
    "tool_name": "list_files",
    "args": {
        "directory_name": "~/Downloads"
    }
}
```
Action result: {'result': ['Leadership-Map', '.DS_Store', '.localized', 'Nvidia-BlueField', 'CNIP-PATHPRIO_Testing-v0001.drawio', 'fortivpn', 'CMP.next_Drawings', 'dcs+-Thor']}
Agent thinking...
Agent response: Now that I have the list of files in the Downloads directory, I need to understand what the user might be asking about. Since there are several files with different extensions and names, I'll proceed by listing them again for clarity before potentially reading any specific file content if requested.

```action
{
    "tool_name": "list_files",
    "args": {
        "directory_name": "~/Downloads"
    }
}
```
Action result: {'result': ['Leadership-Map', '.DS_Store', '.local